In [1]:
%load_ext autoreload
%autoreload 

In [2]:
from dotenv import load_dotenv
load_dotenv() 
from names import DATASET_ID, MODEL_ID
from data import load_db
import os
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.vectorstores import DeepLake
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

embeddings = OpenAIEmbeddings(model=MODEL_ID)
dataset_path = f"hub://{os.environ['ACTIVELOOP_ORG_ID']}/{DATASET_ID}"

db = load_db(dataset_path, embedding_function=embeddings, token=os.environ['ACTIVELOOP_TOKEN'], org_id=os.environ["ACTIVELOOP_ORG_ID"], read_only=True)

/home/zuppif/miniconda3/envs/activeloop/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.4.3) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/zuppif/disney-lyrics



|

hub://zuppif/disney-lyrics loaded successfully.

Deep Lake Dataset in hub://zuppif/disney-lyrics already exists, loading from the storage
Dataset(path='hub://zuppif/disney-lyrics', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape      dtype  compression
  -------   -------   -------    -------  ------- 
 embedding  generic  (85, 1536)  float32   None   
    ids      text     (85, 1)      str     None   
 metadata    json     (85, 1)      str     None   
   text      text     (85, 1)      str     None   


In [80]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from pathlib import Path

prompt = PromptTemplate(
    input_variables=["content"],
    template=Path("prompts/bot.prompt").read_text(),
)

llm = ChatOpenAI(temperature=0.7)

chain = LLMChain(llm=llm, prompt=prompt)

In [81]:
emotions = chain.run(content="It's rainy")
emotions

'Melancholy, Coziness, Nostalgia, Calmness.'

In [84]:
doc, score = db.similarity_search_with_score(emotions, distance_metric="cos")[0]
doc.metadata["embed_url"]

'https://open.spotify.com/embed/track/5EeQQ8BVJTRkp1AIKJILGY?utm_source=generator'

In [83]:
doc.metadata["embed_url"]

from IPython.display import IFrame
IFrame(doc.metadata["embed_url"], width=700, height=350)

In [4]:
db.ds.query("select * where contains(\"text\", 'Did they') limit 2")

Dataset(path='hub://zuppif/disney-lyrics', read_only=True, index=Index([()]), tensors=['embedding', 'ids', 'metadata', 'text'])